In [1]:
import psycopg2
import pandas as pd

# Connect to the Neon DB
connection = psycopg2.connect(
    # Check these parameters
    host="ep-damp-dust-a11bi2qp-pooler.ap-southeast-1.aws.neon.tech", 
    port="5432",
    database="neondb",
    user="neondb_owner",
    password="Ijov0mkf6Ohl",
    sslmode="require"
)

# Create a cursor object
cursor = connection.cursor()


cursor.execute("""
SELECT "ID", "Date", "Count", "Region ID", "Disease ID", "Disease", "Region"
FROM "Kolkata";
""")
kolkata_data = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Date', 'Count', 'Region ID', 'Disease ID', 'Disease', 'Region'])

# Fetch data from Delhi table
cursor.execute("""
SELECT "ID", "Date", "Count", "Region ID", "Disease ID", "Disease", "Region"
FROM "Delhi";
""")
delhi_data = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Date', 'Count', 'Region ID', 'Disease ID', 'Disease', 'Region'])

# Fetch data from Pune table
cursor.execute("""
SELECT "ID", "Date", "Count", "Region ID", "Disease ID", "Disease", "Region"
FROM "Pune";
""")
pune_data = pd.DataFrame(cursor.fetchall(), columns=['ID', 'Date', 'Count', 'Region ID', 'Disease ID', 'Disease', 'Region'])

kolkata_data.to_json('kolkata_data.json', orient='records')
delhi_data.to_json('delhi_data.json', orient='records')
pune_data.to_json('pune_data.json', orient='records')

print("JSON files have been created for Kolkata, Delhi, and Pune data.")
print("Kolkata Data:\n", kolkata_data)
print("\nDelhi Data:\n", delhi_data)
print("\nPune Data:\n", pune_data)

JSON files have been created for Kolkata, Delhi, and Pune data.
Kolkata Data:
     ID       Date  Count  Region ID  Disease ID Disease   Region
0    1   1/1/1900     44          1           1      TB  Kolkata
1    2   1/2/1900    168          1           1      TB  Kolkata
2    3   1/3/1900    146          1           1      TB  Kolkata
3    4   1/4/1900    161          1           1      TB  Kolkata
4    5   1/5/1900    134          1           1      TB  Kolkata
5    6   1/6/1900    108          1           1      TB  Kolkata
6    7   1/7/1900    133          1           1      TB  Kolkata
7    8   1/8/1900    119          1           1      TB  Kolkata
8    9   1/9/1900    112          1           1      TB  Kolkata
9   10  1/10/1900    120          1           1      TB  Kolkata
10  11  1/11/1900     90          1           1      TB  Kolkata
11  12  1/12/1900     87          1           1      TB  Kolkata
12  13  1/13/1900     91          1           1      TB  Kolkata
13  14  1/1

In [2]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Function to create enhanced time series plot using Plotly
def create_enhanced_time_series_plot(df, city_name):
    # Ensure 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date']).dt.day
    
    # Calculate rolling average
    df['Rolling_Avg'] = df['Count'].rolling(window=7).mean()  # 7-day rolling average

    # Create the main line plot
    fig = go.Figure()

    # Add the main line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Count'],
        mode='lines+markers',
        name='Daily Count',
        line=dict(width=2),
        marker=dict(size=6)
    ))

    # Add the rolling average line (7-day rolling)
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Rolling_Avg'],
        mode='lines',
        name='7-day Rolling Average',
        line=dict(width=2, color='red'),
        hoverinfo='skip'
    ))

    # Add a trend line using Plotly's trendline capabilities
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Count'],  # Fit trend using Count vs Date
        mode='lines',
        line=dict(dash='dash', color='green'),
        name='Trend'
    ))

    # Find max and min points
    max_point = df.loc[df['Count'].idxmax()]
    min_point = df.loc[df['Count'].idxmin()]

    # Add annotations for max and min points
    fig.add_trace(go.Scatter(
        x=[max_point['Date']],
        y=[max_point['Count']],
        mode='markers+text',
        text=[f"Max: {max_point['Count']}"],
        textposition="top right",
        marker=dict(color='blue', size=10),
        name='Max Point'
    ))

    fig.add_trace(go.Scatter(
        x=[min_point['Date']],
        y=[min_point['Count']],
        mode='markers+text',
        text=[f"Min: {min_point['Count']}"],
        textposition="bottom right",
        marker=dict(color='orange', size=10),
        name='Min Point'
    ))

    # Customize layout and axis labels
    fig.update_layout(
        title=f'Time Series of Count over Days in {city_name}',
        xaxis_title='Date',
        yaxis_title='Count',
        legend_title='Legend',
        template='plotly_white',  # Clean template with gridlines
        xaxis=dict(
            tickangle=45,
            dtick=1,  # Set a daily tick interval
            showgrid=True
        ),
        yaxis=dict(
            showgrid=True
        ),
        hovermode='x unified'  # Better hover display
    )

    # Show the plot
    fig.show()

# Create plots for each city (replace kolkata_data, delhi_data, pune_data with your actual DataFrames)
create_enhanced_time_series_plot(kolkata_data, 'Kolkata')
create_enhanced_time_series_plot(delhi_data, 'Delhi')
create_enhanced_time_series_plot(pune_data, 'Pune')


In [6]:
import psycopg2
import pandas as pd
import random

# Connect to the Neon DB
connection = psycopg2.connect(
    # Check these parameters
    host="ep-damp-dust-a11bi2qp-pooler.ap-southeast-1.aws.neon.tech", 
    port="5432",
    database="neondb",
    user="neondb_owner",
    password="Ijov0mkf6Ohl",
    sslmode="require"
)

# Create a cursor object
cursor = connection.cursor()

# Fetch data from Drug_Sales table
cursor.execute("""
SELECT "diseaseId", "date", "regionId", "salesCount"
FROM "Drug_Sales";
""")
drug_sales_data = pd.DataFrame(cursor.fetchall(), columns=['diseaseId', 'date', 'regionId', 'salesCount'])

# Convert the 'date' column to datetime format
drug_sales_data['date'] = pd.to_datetime(drug_sales_data['date'])

# Function to assign unique random days in January 1900
def assign_unique_jan_1900_dates(df):
    # Generate a list of unique random days in January 1900
    unique_january_days = pd.date_range('1900-01-01', '1900-01-31').to_list()
    random.shuffle(unique_january_days)
    
    # Get unique dates in the current data
    unique_dates = df['date'].unique()

    # Create a mapping from unique dates to unique random days in January 1900
    date_mapping = {original_date: unique_january_days.pop(0) for original_date in unique_dates}
    
    # Apply the mapping to replace the original dates
    df['date'] = df['date'].map(date_mapping)
    
    return df

# Split the data by region and apply the transformation
region_dfs = {}
unique_regions = drug_sales_data['regionId'].unique()

for region in unique_regions:
    region_df = drug_sales_data[drug_sales_data['regionId'] == region]
    region_dfs[region] = assign_unique_jan_1900_dates(region_df)
    print(f"Modified Data for Region {region}:\n", region_dfs[region])

# Access individual DataFrames by region ID (if needed)
# region_df1 = region_dfs[unique_regions[0]]
# region_df2 = region_dfs[unique_regions[1]]
# region_df3 = region_dfs[unique_regions[2]]


Modified Data for Region 1:
     diseaseId       date  regionId  salesCount
0           1 1900-01-11         1         230
4           2 1900-01-11         1          20
6           1 1900-01-18         1         210
9           2 1900-01-18         1          44
12          1 1900-01-19         1         139
15          2 1900-01-19         1          76
18          1 1900-01-24         1         204
21          2 1900-01-24         1         134
Modified Data for Region 2:
     diseaseId       date  regionId  salesCount
1           2 1900-01-19         2          35
2           1 1900-01-19         2         120
7           1 1900-01-24         2         120
10          2 1900-01-24         2          49
13          1 1900-01-06         2         165
16          2 1900-01-06         2          99
19          1 1900-01-22         2         135
22          2 1900-01-22         2         209
Modified Data for Region 3:
     diseaseId       date  regionId  salesCount
3           1 1900-0

C:\Users\Pranay Thakur\AppData\Local\Temp\ipykernel_9388\3282175675.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Pranay Thakur\AppData\Local\Temp\ipykernel_9388\3282175675.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Pranay Thakur\AppData\Local\Temp\ipykernel_9388\3282175675.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [7]:
import plotly.graph_objs as go
import pandas as pd

# Ensure 'date' column is in datetime format
drug_sales_data['date'] = pd.to_datetime(drug_sales_data['date'])

# Function to create time series plot for drug sales data
def create_drug_sales_time_series(df):
    # Group data by date to get total sales per day
    df_grouped = df.groupby('date').agg({'salesCount': 'sum'}).reset_index()

    # Calculate a rolling average (7-day rolling average)
    df_grouped['Rolling_Avg'] = df_grouped['salesCount'].rolling(window=7).mean()

    # Create the main time series plot
    fig = go.Figure()

    # Add the main sales count line
    fig.add_trace(go.Scatter(
        x=df_grouped['date'],
        y=df_grouped['salesCount'],
        mode='lines+markers',
        name='Daily Sales Count',
        line=dict(width=2),
        marker=dict(size=6)
    ))

    # Add the 7-day rolling average line
    fig.add_trace(go.Scatter(
        x=df_grouped['date'],
        y=df_grouped['Rolling_Avg'],
        mode='lines',
        name='7-day Rolling Average',
        line=dict(width=2, color='red'),
        hoverinfo='skip'
    ))

    # Customize layout and axis labels
    fig.update_layout(
        title='Drug Sales Time Series',
        xaxis_title='Date',
        yaxis_title='Sales Count',
        legend_title='Legend',
        template='plotly_white',  # Clean template with gridlines
        xaxis=dict(
            tickangle=45,
            dtick=86400000.0 * 3,  # Set tick intervals to every 3 days (in milliseconds)
            showgrid=True,
            tickformat="%b %d"  # Format for showing month and day
        ),
        yaxis=dict(
            showgrid=True,
            dtick=10  # Set tick interval for y-axis (adjust based on the scale of your data)
        ),
        hovermode='x unified'  # Better hover display
    )

    # Show the plot
    fig.show()

# Call the function to plot the graph for each region
create_drug_sales_time_series(region_dfs[1])
create_drug_sales_time_series(region_dfs[2])
create_drug_sales_time_series(region_dfs[3])
